In [15]:
import os 
import csv
import pandas  as pd
import numpy as np
from datasets import load_dataset
import requests 
from bs4 import BeautifulSoup
from newspaper import Article, ArticleException
from huggingface_hub import hf_hub_download
import pandas as pd

In [53]:
def getArticleContent(url): 
    
    res = requests.get(url)
    
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # need to figure out how to abstractly find the 'body' of an article 
    body = soup.find('div', class_='main')
    

In [58]:
def getData(csv_file): 
    
    rows = 0
    with open(csv_file) as my_file: 
        
        reader = csv.reader(my_file)
        next(reader) # skip the header 
        
        for row in reader: 
            # each row contains four links 
            # left article, center article, right article, and original page
            # some links are missing
            getArticleContent(row[1])
            break
        


In [59]:
getData('Part1.csv')

None


In [5]:
REPO_ID = "valurank/PoliticalBias"

df1 = pd.read_csv(
    hf_hub_download(repo_id=REPO_ID, filename='Part1.csv', repo_type="dataset")
)
df1 = pd.DataFrame(df1)
df2 = pd.read_csv(
    hf_hub_download(repo_id=REPO_ID, filename='Part2.csv', repo_type="dataset")
)
df2 = pd.DataFrame(df2)
df = df1._append(df2)
df = df.drop(columns='Main URL')
df.to_csv('urls.csv')


In [41]:
for index, row in df.iterrows():
    
    # Iterate over items in the row
    for column, value in row.items():
        if value != np.nan:
            try:
                url = str(value)
                article = Article(url)
                article.download()
                article.parse()
            except ArticleException:
                print('***FAILED TO DOWNLOAD***', url)

In [40]:
REPO_ID = "mediabiasgroup/BABE"

df1 = pd.read_csv(
    hf_hub_download(repo_id=REPO_ID, filename='BABE.csv', repo_type="dataset")
)
df1 = pd.DataFrame(df1)

df1 = df1.dropna()

df1.to_csv('urls_BABE.csv')


In [32]:
df_scraped = pd.DataFrame(columns=['url', 'content', 'type_class'])
x = 0
for index, row in df1.iterrows():
    if(x == 100):
        break
    # Iterate over items in the row
    url = row['news_link']
    url = str(url)
    type_class = row['type']
    if url != np.nan:
        try:
            article = Article(url)
            article.download()
            article.parse()
            df_scraped = df_scraped._append({'url': url, 'content': article.text, 'type_class': type_class}, ignore_index=True)
            
        except ArticleException:
                print('***FAILED TO DOWNLOAD***', url)
    x +=1
    
print(df_scraped.head(20))

***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/breitbart/~3/KernUHEqyEk/
***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/breitbart/~3/hmjqwWAvPXQ/
***FAILED TO DOWNLOAD*** https://www.reuters.com/article/us-usa-election-james/nba-star-lebron-james-emerges-as-potent-political-force-ahead-of-november-u-s-election-idUSKBN25O1B10
***FAILED TO DOWNLOAD*** https://www.reuters.com/article/us-global-race-usa-wisconsin/arsonists-set-fires-as-wisconsin-city-rages-anew-over-police-shooting-of-black-man-idUSKBN25L0Q9?utm_medium=Social&utm_source=twitter
***FAILED TO DOWNLOAD*** https://www.reuters.com/article/us-global-race-usa-los-angeles/l-a-sheriffs-deputies-fatally-shoot-black-man-after-suspected-bike-violation-idUSKBN25S606
***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/breitbart/~3/B0iqqthw_uw/
***FAILED TO DOWNLOAD*** https://www.reuters.com/investigates/special-report/usa-police-rochester-shooting/
***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/breitbart/

In [37]:
def relable_class(row):
    if row['type_class'] == 'left':
        return 0
    elif row['type_class'] == 'center':
        return 1
    elif row['type_class'] == 'right':
        return 2
    else:
        return row['type_class']  # Return the original value if not one of the specified types


In [39]:
df_scraped['type_class'] = df_scraped.apply(relable_class, axis=1)
df_scraped.to_csv('BABE_scraped.csv')
type_counts = df_scraped['type_class'].value_counts() # eval number of each type_class
print(type_counts)

type_class
0    30
2    24
1     5
Name: count, dtype: int64
